In [ ]:
import Pkg
Pkg.activate("MyProject")
Pkg.develop(path="/home/silicon/Documents/Git/StockFlow.jl/")

Pkg.instantiate()

In [ ]:
using Revise

In [ ]:
using GraphViz

using StockFlow
using StockFlow.Syntax

using Catlab
using Catlab.CategoricalAlgebra
using LabelledArrays
using OrdinaryDiffEq
using Plots

using Catlab.Graphics
using Catlab.Programs
using Catlab.Theories
using Catlab.WiringDiagrams

This model is re-created based on Garnett's paper:
https://journals.lww.com/stdjournal/Fulltext/2000/11000/Epidemiology_and_Control_of_Curable_Sexually.7.aspx

# 1. Define the components of all the sub-models
## 1.1 The births and deaths of Stock X

In [ ]:
openX = Open( 
    (@stock_and_flow begin

        :stocks
        X

        :parameters
        μ

        :dynamic_variables
        v_births = N * μ
        v_deaths = X * μ

        :flows
        CLOUD => f_births(v_births) => X
        X => f_deathX(v_deaths) => CLOUD

        :sums
        N = [X]

        end 
    ),
    # feet
    @foot X => N
    #foot(:X,:N,:X=>:N)
    
)



In [ ]:
Graph(apex(openX))

## 1.2 The SIS sub-model structure

In [ ]:
openSIS_A = Open(
    (@stock_and_flow begin

        :stocks
        X
        A

        :parameters
        μ
        ϕ_complement
        σ′
        cβ

        :dynamic_variables
        v_newInfectiousA₁ = ϕ_complement * X
        v_newInfectiousA₂ = v_newInfectiousA₁ * cβ
        v_newInfectiousA₃ = v_newInfectiousA₂ * NI
        v_newInfectiousA₄ = v_newInfectiousA₃ / N

        v_deathA = A * μ

        v_recoveryA = A * σ′

        :flows
        X => f_newInfectiousA(v_newInfectiousA₄) => A
        A => f_deathA(v_deathA) => CLOUD
        A => f_recoveryA(v_recoveryA) => X

        :sums
        N = [X, A]
        NI = [A]

    end
    ),
     # feet
    @foot X => N,
     foot(:X,:N,:X=>:N),
     foot((),:NI,())

)

In [ ]:
Graph(apex(openSIS_A))

In [ ]:
openSIS_Y = Open(
    (@stock_and_flow begin
    
    :stocks
    X
    Y

    :parameters
    μ
    ϕ
    σ
    cβ

    :dynamic_variables
    v_newInfectiousY₁ = ϕ * X
    v_newInfectiousY₂ = v_newInfectiousY₁ * cβ
    v_newInfectiousY₃ = v_newInfectiousY₂ * NI
    v_newInfectiousY₄ = v_newInfectiousY₃ / N

    v_deathY = Y * μ

    v_recoveryY = Y * σ

    :flows
    X => f_newInfectiousY(v_newInfectiousY₄) => Y
    Y => f_deathY(v_deathY) => CLOUD
    Y => f_recoveryY(v_recoveryY) => X

    :sums
    N = [X, Y]
    NI = [Y]

end
        ),

     # feet
     foot(:X,:N,:X=>:N),
     foot((),:NI,())
    
)

In [ ]:
Graph(apex(openSIS_Y))

# 2 Compose
## 2.1 Define composition rule

In [ ]:
# define the UWD-algebra
uwd = @relation (XN,NI) begin
    X(XN)
    XA(XN,NI)
    XY(XN,NI)
end;
display_uwd(uwd)

In [ ]:
XAY_model = oapply(uwd,Dict(
    :X=>openX,
    :XA=>openSIS_A,
    :XY=>openSIS_Y
    )) |> apex
Graph(XAY_model)

In [ ]:
Graph(XAY_model,type="SF")

# 3. Solve ODEs

In [ ]:
# define values of constant parameters
# unit: year
p = LVector(
    cβ=15*0.7, μ=1/15,ϕ=(0.95+0.4)/2,σ=12/4,σ′=12/4,ϕ_complement=(1.0 - ((0.95+0.4)/2))
)
# define initial values for stocks
u0 = LVector(
    X=990, A=5, Y=5
)

In [ ]:
# results are tested the same as the Anylogic model
prob_XAY = ODEProblem(vectorfield(XAY_model),u0,(0.0,2.0),p);
sol = solve(prob_XAY,Tsit5(),abstol=1e-8);
plot(sol)

In [ ]:
# to have the figures plotted fix to the wider of the cells
HTML("""
<style>
.output_svg div{
  width: 100% !important;
  height: 100% !important;
}
</style>
""")
